# Steam Trike Calculations
Like many design tasks, sizing the various components of the steam trike is an iterative process. We start with certain assumed parameters and goals, then develop cylinder and accumulator dimensions. These, in turn, affect piping and generator design. At various points in the design, we can compare the results against the initial assumptions and adjust.

In this notebook, we start with one set of assumed parameters as a demonstration, but develop the code in a way that we can evaluate different sets of parameters easily.

In [1]:
# initial setup
%pip install pint
%pip install pyXSteam
import numpy as np
import pandas as pd
import pint                                 # units library
from pyntXSteam import pyntXSteam           # pint wrapper for steam library pyXSteam
ureg = pint.UnitRegistry()                  # initialize pint units registry for this notebook
ureg.autoconvert_offset_to_baseunit = True  # disable relative temperature units
ureg.default_format = "~P"                  # enable pretty unit outputs
steam = pyntXSteam(ureg)                    # initialize steam library with pint registry
x = {}                                      # iteration parameters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Initial Parameters

In [2]:
x['trike_weight'] = 2000 * ureg('lb')           # Based on homebuilt electric and CE trikes coming in at 800-1500 lb online
x['avg_pressure'] = 125 * ureg('lbf/in**2')     # Based on max of 150 psi saturated steam with margin
x['pressure_deadband'] = 40 * ureg('lbf/in**2') # Based on avg_pressure and max 150 psi saturated steam, with margen
x['max_velocity'] = 55 * ureg('mi/hr')          # Based on a minimum speed for highway but not freeway travel

## Required Wheel Diameter
What's the maximum RPM we can expect to be able to achieve with the reciprocating engine? Common practice says 250-500 RPM, but we can check top RPMs of prototypical locomotives and steam cars.

| Prototype | Max Speed (mi/hr) | Driver Diameter (in) | rev/min |
| --- | --- | --- | --- |
| UP 844 | 120 | 80 | 504 |
| UP 4014 | 80 | 68 | 395 |
| Reading 2102 | 80 | 70 | 384 |
| PM 1225 | 70 | 69 | 341 |
| N&W 611 | 110 | 70 | 528 |
| SP 4449 | 110 | 80 | 462 |
| Stanley Steam Car | 60 | (N/A) | 900 |
| Doble Steam Car | 70 | (N/A) | 900 |

In [3]:
x['max_rpm'] = 350 * ureg('1/min')           # Conservative bet for larger reciprocating steam engine

With maximum RPM and maximum velocity, we can calculate the wheel diameter.

In [4]:
def wheel_dia(y):
    value = y['max_velocity'] / y['max_rpm'] / np.pi

    value = value.to('in')
    y['wheel_dia'] = value
    return value
wheel_dia(x)

52.821251970155906 <Unit('inch')>

## Required Acceleration
Let's try two paths I have thought of to come up with a target maximum acceleration.

### Normal Acceleration Data
We want acceleration behavior compatible with traffic, so we can look at published data for conventional road vehicles. The data shown in Figures 1 and 2 are for consumer vehicles accelerating in normal traffic, not the maximum acceleration the vehicles are capable of. Thus, we want to target a value on the high end to reflect extra capacity available for "flooring it".


<div align="center">
<img src="fig5e.PNG" width="400"/><br />
Fig 1. Acceleration vs speed of various gasoline vehicles. Graph, "Scatter plots of acceleration-speed," from P.S.Bokare and A.K.Maurya, Acceleration-Deceleration Behaviour of Various Vehicle Types, page 10, figure 5(e).
</div><br />
<div align="center">
<img src="fig6e.PNG" width="400"/><br />
Fig 2. Idealized acceleration vs speed of gasoline vehicles. Graph, "Idealized plot of acceleration with speed for all vehicle types," from P.S.Bokare and A.K.Maurya, Acceleration-Deceleration Behaviour of Various Vehicle Types, page 11, figure 6(e).
</div><br />


The acceleration curves for these conventional vehicles include gear shifts and will not represent the curve for the steam trike. Instead, we are looking to have a comparable acceleration from a stop.

Candidate `max_accel` = 2 \[m/s^2\] ≅ 6.5 \[ft/s^2\]

### 0-60 MPH Time Data
0-60 MPH times for vehicles represent maximum acceleration, which is what we want. However, because car acceleration varies as the car accelerates, you're looking at an average max acceleration instead of the highest instantaneous acceleration. Data from ZeroTo60Times.com.

| Prototype | 0-60 MPH Time (s) | Acceleration (m/s^2) |
| --- | --- | --- |
| Geo Metro | 12.6 | 7.0 |
| Nissan Versa | 9.0 | 10.0 |
| Honda Fit | 7.8 | 11.3 |

### Selection
We see that the candidate max_accel from normal vehicle acceleration is lower than even the Geo Metro's 0-60 MPH, which makes sense as that was normal traffic acceleration instead of max. Given that we are looking for a usable max acceleration target, let's use that of the Geo Metro.

In [5]:
x['max_accel'] = 7.0 * ureg('ft/s**2')

## Cylinder Dimensions
### Required Force at Roadway
Required force on the road to accelerate `trike_weight` by `max_accel`

In [6]:
def road_force(y):
    value = y['trike_weight'] * y['max_accel']

    value = value.to('lbf')
    y['road_force'] = value
    return value
road_force(x)

435.13330240194153 <Unit('force_pound')>

### Required Cylinder Volume
The mean tractive force of a two cylinder engine, at 90 percent cutoff, with the cylinders 90 degrees out of phase is given by:

F = K2 * K3 * avg_pressure * cyl_volume / wheel_dia
* K2 = 1.2 - The ratio of mean torque of two identical piston-crank assemblies 90 degrees out of phase to the peak torque of one of the piston-crank assemblies (Jeffrey Hook, Fundamentals of Steam Locomotive Tractive Force, page 7)
* K3 = 0.9 - A fudge factor to account for pressure loss between the boiler and the cylinder (Hook)
* cyl_volume - The swept volume of the cylinder

We will solve for the swept volume of the cylinder,

In [7]:
def cyl_volume(y):
    value = road_force(y) * wheel_dia(y) * 1.2 * 0.9 / y['avg_pressure']

    value = value.to('in**3')
    y['cyl_volume'] = value
    return value
cyl_volume(x)

198.58422937057054 <Unit('inch ** 3')>

In [8]:
def max_hp(y):
    value = road_force(y) * wheel_dia(y) * np.pi * y['max_rpm']

    value = value.to('hp')
    y['road_force'] = value
    return value
max_hp(x)

63.819551018951444 <Unit('horsepower')>

### Required Cylinder Dimensions
We set cylinder bore as a parameter and calculate stroke and max piston speed during stroke, which is needed for seal selection later.

In [9]:
x['cyl_bore'] = 5.0 * ureg('in')
def cyl_stroke(y):
    value = cyl_volume(y) / (y['cyl_bore']**2 * np.pi / 4)

    value = value.to('in')
    y['cyl_stroke'] = value
    return value
cyl_stroke(x)

10.113811751814735 <Unit('inch')>

In [10]:
def max_piston_speed(y):
    value = y['max_rpm'] * cyl_stroke(y) / 2

    value = value.to('ft/s')
    y['max_piston_speed'] = value
    return value
max_piston_speed(x)


2.4582181341216365 <Unit('foot / second')>

## Steam System Sizing
### Maximum Steam Rate
Maximum steam rate occurs at max power output, so we need to define a reasonable target. One candidate for max power condition is full power at full throttle and full cutoff, but that is overly conservative. Insetad, lets define max power as the power required to go up a target slope at full speed.

In the vicininity of Austin, Ranch Road 2222 encounters a hill of 10% slope for most of a mile as shown in Figure 3. Not far from there, Figure 4 shows that Spicewood Springs has a 15% slope for ~500 ft. Many vehicles struggle to maintain speed going up that slope. Let's design for the longer slope and allow the energy in the accumulator to handle short bursts like Spicewood Springs.

<div align="center">
<img src="2222slope.PNG" width="400"/><br />
Fig 3. Google Earth Pro elevation profile for RM 2222 between Loop 360 and FM 620.
</div><br />
<div align="center">
<img src="spicewoodslope.PNG" width="400"/><br />
Fig 4. Google Earth Pro elevation profile for Spicewood Springs Road near Loop 360.
</div><br />

We neglect air resistance for this calculation, and assume the throttle is full open but cutoff has been linked up. This means that on each stroke, a certain volume of steam is admitted at the full pressure available to the cylinder, then allowed to expand to an exhaust pressure. We can assume isentropic expansion to find the mass of steam required to produce the energy per stroke required to maintain speed up the target slope.

In [11]:

def max_steam_mass_rate(y):
    x['max_road_slope'] = 0.10
    slope_rad = np.arctan(x['max_road_slope'])
    # convert trike_weight to force (assume Earth)
    slope_force = (y['trike_weight'].to('lb').magnitude * ureg('lbf')) * np.sin(slope_rad)
    slope_power = slope_force * y['max_velocity']
    stroke_energy = slope_power / y['max_rpm'] / 4
    # convert avg_pressure into absolute pressure, 
    # taking into account pressure loss between boiler and cylinder per Hook's approximation
    p1 = y['avg_pressure'] * 0.9 + (1 * ureg('atm'))
    p2 = 1 * ureg('atm') + 10 * ureg('lbf/in**2')
    u1 = steam.uV_p(p1)
    # assume isentropic expansion
    s = steam.sV_p(p1)
    u2 = steam.u_ps(p2, s)
    stroke_mass = stroke_energy / (u1 - u2)
    value =  stroke_mass * 4 * y['max_rpm']
    
    value = value.to('lb/min')
    y['max_steam_mass_rate'] = value
    return value
max_steam_mass_rate(x)

11.416322302878683 <Unit('pound / minute')>

In [12]:
def max_steam_liquid_rate(y):
    value = max_steam_mass_rate(y) * steam.v_pt(1 * ureg('atm'), 25 * ureg('degC'))
    
    value = value.to('gal/min')
    y['mass_steam_liquid_rate'] = value
    return value
max_steam_liquid_rate(x)

1.3720272953619692 <Unit('gallon / minute')>

### Accumulator Sizing
We can use the mass flow at average boiler pressure to determine how long the accumulator will last at max steam rate when discharging from the top to the bottom of the pressure deadband.

In [13]:
x['accum_capacity'] = 15 * ureg('gal')
def accum_steam_mass(y):
    max_p = y['avg_pressure'] + y['pressure_deadband'] / 2
    min_p = y['avg_pressure'] - y['pressure_deadband'] / 2
    # assume accumulator is 90% full of saturated water when charged
    liquid_volume = x['accum_capacity'] * 0.9
    liquid_mass = liquid_volume / steam.vL_p(max_p)
    # using equation 3.22.1 from the Spirax Sarco guide
    delta_h = steam.hL_p(max_p) - steam.hL_p(min_p)
    value = delta_h * liquid_mass / (steam.hV_p(min_p) - steam.hL_p(min_p))

    value = value.to('lb')
    y['accum_steam_mass'] = value
    return value
accum_steam_mass(x)

2.8875662456430393 <Unit('pound')>

In [14]:
def accum_buffer_time(y):
    value = accum_steam_mass(y) / max_steam_mass_rate(y)

    value = value.to('s')
    y['accum_buffer_time'] = value
    return value
accum_buffer_time(x)

15.17598839119105 <Unit('second')>

The Spirax Sarco [accumulator sizing guide](https://www.spiraxsarco.com/learn-about-steam/the-boiler-house/steam-accumulators) includes an empirical relationship between available saturated water surface area, pressure, and the maximum steam release rate in an accumulator without entraining water. We use this to find the required surface area in the accumulator required to support max steam rate.

In [15]:

def accum_surface_area(y):
    area = max_steam_mass_rate(y).to('kg/hr').magnitude / 220 / (y['avg_pressure'].to('bar').magnitude + 1)
    value = (area * ureg('m**2'))

    value = value.to('ft**2')
    y['accum_surface_area'] = value
    return value
accum_surface_area(x)

1.5804678543993678 <Unit('foot ** 2')>

### Steam Generator Sizing
Next, we find the heat input required to boil water that water (at constant pressure, as the feed pump/injector handles that), assuming some heat transfer efficiency for the generator coil.

In [16]:

x['burner_power'] = 700000 * ureg('Btu/hr')
x['gen_coil_efficiency'] = 0.7
def gen_mass_rate(y):
    init_h = steam.h_pt(y['avg_pressure'] + 1 * ureg('atm'), 70 * ureg('degF'))   # liquid phase
    final_h = steam.hV_p(y['avg_pressure'] + 1 * ureg('atm'), 1 * ureg(''))    # dry vapor phase
    heat_transfer_rate = x['burner_power'] * x['gen_coil_efficiency']
    value = heat_transfer_rate / (final_h - init_h)

    value = value.to('lb/min')
    y['gen_mass_rate'] = value
    return value
gen_mass_rate(x)

7.071540885082615 <Unit('pound / minute')>

With the average generator production, we can estimate how long it takes to charge the accumulator from "fully discharged".

In [17]:
def accum_charge_time(y):
    value = accum_steam_mass(y) / gen_mass_rate(y)

    value = value.to('s')
    y['accum_charge_time'] = value
    return value
accum_charge_time(x)


24.500172954392568 <Unit('second')>

### TODO: Generator Coil Dimensions

## Evalution of Initial Iteration
Now that we have calculated a single design iteration, let's evaluate the results.

In [18]:
df = pd.DataFrame(x, index=[1])
pd.set_option('display.max_columns', None)
df

,trike_weight,avg_pressure,pressure_deadband,max_velocity,max_rpm,wheel_dia,max_accel,road_force,cyl_volume,cyl_bore,cyl_stroke,max_piston_speed,max_road_slope,max_steam_mass_rate,mass_steam_liquid_rate,accum_capacity,accum_steam_mass,accum_buffer_time,accum_surface_area,burner_power,gen_coil_efficiency,gen_mass_rate,accum_charge_time
1,2000 lb,125.0 lbf/in²,40.0 lbf/in²,55.0 mi/h,350.0 1/min,52.821251970155906 in,7.0 ft/s²,435.13330240194153 lbf,198.58422937057054 in³,5.0 in,10.113811751814735 in,2.4582181341216365 ft/s,0.1,11.416322302878683 lb/min,1.3720272953619692 gal/min,15 gal,2.8875662456430393 lb,15.17598839119105 s,1.5804678543993678 ft²,700000.0 Btu/h,0.7,7.071540885082615 lb/min,24.500172954392568 s


* Wheel diameter - Road legal tires up to 53" are commercially available, but they are heavy, expensive, and wide
* Cylinder dimensions - Performing any machining operations on the 10 inch axis of the cylinder block is going to be difficult on any readily-available machine, but finding and working with stock that supports a 5 inch or larger bore will be difficult too
* Max Steam Heat Rate - 1.5 million BTU/hr is over twice what the largest DC burner available can put out, and that's not even including the inefficiency of heat transfer to the coil

From here, we can tweak assumptions as we like to get a design that looks more reasonable.

# Iterations

In [19]:
iters = [
    x,  # 0: initial iteration from text
    {   # 1: lighter, slower
        "trike_weight": 1500 * ureg('lb'),              # 2000 -> 1500 lb
        "avg_pressure": 125 * ureg('lbf/in**2'),
        "pressure_deadband": 40 * ureg('lbf/in**2'),
        "max_velocity": 45 * ureg('mi/hr'),             # 55 -> 45 MPH
        "max_rpm": 350 * ureg('1/min'),
        "max_accel": 7 * ureg('ft/s**2'),
        "cyl_bore": 5.0 * ureg('in'),
        "max_road_slope": 0.1 * ureg(''),
        "accum_capacity": 15 * ureg('gal'),
        "gen_coil_efficiency": 0.7,
        "burner_power": 700000 * ureg('Btu/hr')
    },
    {   # 2: lighter, slower, lower avg pressure, bigger pressure deadband
        "trike_weight": 1500 * ureg('lb'),              # 2000 -> 1500 lb
        "avg_pressure": 110 * ureg('lbf/in**2'),        # 125 -> 110 psi
        "pressure_deadband": 70 * ureg('lbf/in**2'),    # 40 -> 70 psi
        "max_velocity": 45 * ureg('mi/hr'),             # 55 -> 45 psi
        "max_rpm": 350 * ureg('1/min'),
        "max_accel": 7 * ureg('ft/s**2'),
        "cyl_bore": 5.0 * ureg('in'),
        "max_road_slope": 0.1 * ureg(''),
        "accum_capacity": 15 * ureg('gal'),
        "gen_coil_efficiency": 0.7,
        "burner_power": 700000 * ureg('Btu/hr')
    },
    {   # 3: lighter, higher rpm
        "trike_weight": 1500 * ureg('lb'),              # 2000 -> 1500 lb
        "avg_pressure": 125 * ureg('lbf/in**2'),        
        "pressure_deadband": 40 * ureg('lbf/in**2'),
        "max_velocity": 55 * ureg('mi/hr'),
        "max_rpm": 450 * ureg('1/min'),                 # 350 -> 450 RPM
        "max_accel": 7 * ureg('ft/s**2'),
        "cyl_bore": 5.0 * ureg('in'),
        "max_road_slope": 0.1 * ureg(''),
        "accum_capacity": 15 * ureg('gal'),
        "gen_coil_efficiency": 0.7,
        "burner_power": 700000 * ureg('Btu/hr')
    },
    {   # 4: lighter, higher rpm, lower avg pressure, bigger pressure deadband
        "trike_weight": 1500 * ureg('lb'),              # 2000 -> 1500 lb
        "avg_pressure": 110 * ureg('lbf/in**2'),        # 125 -> 110 psi
        "pressure_deadband": 70 * ureg('lbf/in**2'),    # 40 -> 70 psi
        "max_velocity": 55 * ureg('mi/hr'),
        "max_rpm": 450 * ureg('1/min'),                 # 350 -> 450
        "max_accel": 7 * ureg('ft/s**2'),
        "cyl_bore": 5.0 * ureg('in'),
        "max_road_slope": 0.1 * ureg(''),
        "accum_capacity": 15 * ureg('gal'),
        "gen_coil_efficiency": 0.7,
        "burner_power": 700000 * ureg('Btu/hr')
    }   
]

In [20]:
# run calculations
for each in iters:
    max_hp(each)
    max_piston_speed(each)
    max_steam_liquid_rate(each)
    accum_buffer_time(each)
    accum_surface_area(each)
    accum_charge_time(each)
# display
df = pd.DataFrame(iters)
df

,trike_weight,avg_pressure,pressure_deadband,max_velocity,max_rpm,wheel_dia,max_accel,road_force,cyl_volume,cyl_bore,cyl_stroke,max_piston_speed,max_road_slope,max_steam_mass_rate,mass_steam_liquid_rate,accum_capacity,accum_steam_mass,accum_buffer_time,accum_surface_area,burner_power,gen_coil_efficiency,gen_mass_rate,accum_charge_time
0,2000 lb,125.0 lbf/in²,40.0 lbf/in²,55.0 mi/h,350.0 1/min,52.821251970155906 in,7.0 ft/s²,435.13330240194153 lbf,198.58422937057054 in³,5.0 in,10.113811751814735 in,2.4582181341216365 ft/s,0.1,11.416322302878683 lb/min,1.3720272953619692 gal/min,15 gal,2.8875662456430393 lb,15.17598839119105 s,1.5804678543993678 ft²,700000.0 Btu/h,0.7,7.071540885082615 lb/min,24.500172954392568 s
1,1500 lb,125.0 lbf/in²,40.0 lbf/in²,45.0 mi/h,350.0 1/min,43.2173879755821 in,7.0 ft/s²,326.34997680145614 lbf,121.85850438648647 in³,5.0 in,6.206202665886314 in,1.5084520368473677 ft/s,0.1,7.0054705040391925 lb/min,0.8419258403357539 gal/min,15 gal,2.8875662456430393 lb,24.731240341200227 s,0.9698325470177939 ft²,700000.0 Btu/h,0.7,7.071540885082615 lb/min,24.500172954392568 s
2,1500 lb,110.0 lbf/in²,70.0 lbf/in²,45.0 mi/h,350.0 1/min,43.2173879755821 in,7.0 ft/s²,326.34997680145614 lbf,138.47557316646188 in³,5.0 in,7.052503029416266 in,1.714150041872009 ft/s,0.1,7.517550075705447 lb/min,0.9034681769204705 gal/min,15 gal,5.555652216478453 lb,44.34145827188609 s,1.1661091853372991 ft²,700000.0 Btu/h,0.7,7.083060417041882 lb/min,47.061455551994364 s
3,1500 lb,125.0 lbf/in²,40.0 lbf/in²,55.0 mi/h,450.0 1/min,41.08319597678793 in,7.0 ft/s²,326.34997680145614 lbf,115.84080046616616 in³,5.0 in,5.899723521891929 in,1.8436636005912275 ft/s,0.1,8.562241727159012 lb/min,1.029020471521477 gal/min,15 gal,2.8875662456430393 lb,20.23465118825473 s,1.1853508907995256 ft²,700000.0 Btu/h,0.7,7.071540885082615 lb/min,24.500172954392568 s
4,1500 lb,110.0 lbf/in²,70.0 lbf/in²,55.0 mi/h,450.0 1/min,41.08319597678793 in,7.0 ft/s²,326.34997680145614 lbf,131.637273257007 in³,5.0 in,6.704231274877191 in,2.095072273399122 ft/s,0.1,9.188116759195545 lb/min,1.1042388829027971 gal/min,15 gal,5.555652216478453 lb,36.27937494972499 s,1.4252445598566987 ft²,700000.0 Btu/h,0.7,7.083060417041882 lb/min,47.061455551994364 s
